# Session Discovery

이 노트북은 오프라인 평가를 위해 AgentCore Observability에서 agent 세션을 검색합니다. agent의 trace 로그 그룹을 쿼리하여 세션을 찾은 다음, 분석 노트북에서 처리할 수 있도록 JSON 파일로 저장합니다.

**두 가지 검색 방법:**

1. **시간 기반**: 시간 범위 내의 모든 세션을 찾습니다. 최근 agent 활동의 대량 평가에 사용합니다.

2. **점수 기반**: AgentCore의 기존 평가 점수로 세션을 찾습니다. 업데이트된 rubric으로 낮은 점수의 세션을 재평가하는 데 사용합니다.

**출력:** 분석 노트북을 위한 세션 ID와 메타데이터가 포함된 `discovered_sessions.json`

## 워크플로우에서의 위치

이것은 평가 워크플로우의 **노트북 1**입니다. 여기서 세션을 검색한 후, 두 가지 평가 경로 중 하나를 선택합니다.

![Notebook Workflow](images/notebook_workflow.svg)

## Setup

필요한 모듈을 import하고 `config.py`에서 설정을 로드합니다. 모든 설정 값은 이 셀을 실행하기 전에 환경 변수를 통해 재정의할 수 있습니다.

In [ ]:
import logging
import sys
from datetime import datetime, timedelta, timezone

# 현재 디렉토리를 모듈 검색 경로에 추가
sys.path.insert(0, ".")

# config.py에서 설정값 로드
from config import (
    AWS_REGION,
    SOURCE_LOG_GROUP,
    EVAL_RESULTS_LOG_GROUP_FULL,
    LOOKBACK_HOURS,
    MAX_SESSIONS,
    MIN_SCORE,
    MAX_SCORE,
    DISCOVERED_SESSIONS_PATH,
    EVALUATOR_NAME,
)

# CloudWatch Logs 쿼리 및 세션 검색 유틸리티
from utils import (
    ObservabilityClient,
    SessionDiscoveryResult,
    # SessionInfo,
)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

## Configuration

점수 기반 검색에 사용되는 `EVALUATOR_NAME`은 `config.py`에서 로드됩니다. 이는 기존 평가 결과의 evaluator 이름과 일치해야 합니다. `LOOKBACK_HOURS`, `MAX_SESSIONS` 또는 점수 임계값과 같은 설정을 변경하려면 `config.py`를 수정하세요.

In [ ]:
# EVALUATOR_NAME is loaded from config.py
print(f"Using evaluator: {EVALUATOR_NAME}")

## Initialize Client

CloudWatch Logs Insights 쿼리를 처리하는 `ObservabilityClient`를 생성합니다. 시간 범위는 config의 `LOOKBACK_HOURS`에서 계산됩니다.

In [ ]:
# CloudWatch Logs Insights 쿼리를 위한 클라이언트 초기화
obs_client = ObservabilityClient(
    region_name=AWS_REGION,
    log_group=SOURCE_LOG_GROUP,
)

# 시간 범위 계산 (현재 시각부터 LOOKBACK_HOURS 이전까지)
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(hours=LOOKBACK_HOURS)
# CloudWatch Logs API는 밀리초 단위 timestamp 사용
start_time_ms = int(start_time.timestamp() * 1000)
end_time_ms = int(end_time.timestamp() * 1000)

## Time-Based Discovery

시간 범위 내의 모든 고유한 세션 ID에 대해 AgentCore Observability 로그 그룹을 쿼리합니다. span 수와 타임스탬프가 포함된 세션을 반환하며, 가장 최근 활동 순으로 정렬됩니다. 최근의 모든 agent 상호작용을 평가하려는 경우 이 방법을 사용하세요.

In [ ]:
# 시간 범위 내의 모든 세션 검색 (최근 활동 순으로 정렬)
time_based_sessions = obs_client.discover_sessions(
    start_time_ms=start_time_ms,
    end_time_ms=end_time_ms,
    limit=MAX_SESSIONS,
)

print(f"Discovered {len(time_based_sessions)} sessions")

## Score-Based Discovery

AgentCore 평가 결과 로그 그룹을 쿼리하여 기존 평가 점수로 세션을 찾습니다. 지정된 evaluator가 `MIN_SCORE`와 `MAX_SCORE` 사이의 점수를 매긴 세션을 필터링합니다. 업데이트된 rubric으로 성능이 낮은 세션을 재평가하려는 경우 이 방법을 사용하세요.

In [ ]:
# 평가 점수 기반으로 세션 검색 (MIN_SCORE ~ MAX_SCORE 범위)
score_based_sessions = obs_client.discover_sessions_by_score(
    evaluation_log_group=EVAL_RESULTS_LOG_GROUP_FULL,
    evaluator_name=EVALUATOR_NAME,
    start_time_ms=start_time_ms,
    end_time_ms=end_time_ms,
    min_score=MIN_SCORE,
    max_score=MAX_SCORE,
    limit=MAX_SESSIONS,
)

print(f"Discovered {len(score_based_sessions)} sessions by score")

## Select Discovery Method

사용할 검색된 세션 세트를 선택합니다. 시간 기반 결과를 사용하려면 `USE_TIME_BASED = True`로 설정하고, 점수 기반 결과를 사용하려면 `False`로 설정합니다. 선택된 세션은 검색 방법에 대한 메타데이터와 함께 `SessionDiscoveryResult`로 패키징됩니다.

In [ ]:
# Set to False to use score-based discovery instead
USE_TIME_BASED = True

# 검색 방법에 따라 세션 선택 및 메타데이터 설정
if USE_TIME_BASED:
    selected_sessions = time_based_sessions
    discovery_method = "time_based"
    log_group = SOURCE_LOG_GROUP
    filter_criteria = None
else:
    selected_sessions = score_based_sessions
    discovery_method = "score_based"
    log_group = EVAL_RESULTS_LOG_GROUP_FULL
    filter_criteria = {
        "evaluator_name": EVALUATOR_NAME,
        "min_score": MIN_SCORE,
        "max_score": MAX_SCORE,
    }

# 검색 결과를 메타데이터와 함께 패키징
discovery_result = SessionDiscoveryResult(
    sessions=selected_sessions,
    discovery_time=datetime.now(timezone.utc),
    log_group=log_group,
    time_range_start=start_time,
    time_range_end=end_time,
    discovery_method=discovery_method,
    filter_criteria=filter_criteria,
)

print(f"Selected {len(selected_sessions)} sessions via {discovery_method}")

## Preview Sessions

처음 10개의 검색된 세션을 확인합니다. 시간 기반 검색의 경우 세션 ID와 span 수를 표시합니다. 점수 기반 검색의 경우 세션 ID와 평균 평가 점수를 표시합니다.

In [ ]:
# 처음 10개 세션 미리보기
for i, session in enumerate(selected_sessions[:10]):
    meta = session.metadata or {}
    if discovery_method == "time_based":
        print(f"{i+1}. {session.session_id} - {session.span_count} spans")
    else:
        print(f"{i+1}. {session.session_id} - avg_score: {meta.get('avg_score', 0):.2f}")

## Save Results

검색 결과를 JSON으로 저장합니다. 이 파일은 각 세션을 처리하기 위해 분석 노트북에서 로드됩니다. 출력 경로는 `config.py`에서 `DISCOVERED_SESSIONS_PATH`로 설정됩니다.

In [ ]:
# 검색 결과를 JSON 파일로 저장 (분석 노트북에서 사용)
discovery_result.save_to_json(DISCOVERED_SESSIONS_PATH)
print(f"Saved {len(selected_sessions)} sessions to {DISCOVERED_SESSIONS_PATH}")

## Verify Output

JSON 파일이 올바르게 저장되었는지 확인합니다. 확인 후, 이러한 세션을 평가하기 위해 멀티 세션 분석 노트북으로 진행하세요.

In [ ]:
import json

# 저장된 JSON 파일 검증
with open(DISCOVERED_SESSIONS_PATH, "r") as f:
    saved_data = json.load(f)

print(f"Sessions: {len(saved_data['sessions'])}")
print(f"Method: {saved_data['discovery_method']}")
print(f"Time range: {saved_data['time_range_start']} to {saved_data['time_range_end']}")